In [1]:
import os
import pandas as pd
import mysql.connector as msql
from dotenv import load_dotenv

load_dotenv()

host = os.getenv('host')
user = os.getenv('user')
password = os.getenv('password')
database = os.getenv('database')

db = msql.connect(host=host, user=user, password=password, database=database)
cursor = db.cursor(buffered=True)
#display(cursor)

In [2]:
company = 'CosmoBio'
filename = 'AMSbio_Price_List_Active_Oct_1_2023.xlsx'

In [3]:
price_file = pd.ExcelFile("C://Users//Shri//AMSBio//AMSBIO Sharepoint - Documents//ShareDocs//Current Price_lists//2023//{0}//{1}".format(company, filename))
# price_file = pd.ExcelFile("G://Current Price_lists//2022//{0}//{1}".format(company, filename))
sheet_names = list(price_file.sheet_names)
#print(sheet_names)
sheet1 = price_file.parse(sheet_names[0])

In [2]:
sheet1.head(3)

NameError: name 'sheet1' is not defined

In [5]:
def fetchCategory(code, column, cursor):    
    cursor.execute(f"SELECT category_1 FROM product_records WHERE {column} LIKE '{code}'")
    result = cursor.fetchone()
    if result != None:
        return result[0]
    else:
        return "NA"

In [6]:
sheet1['Lowest_Node'] = 0

In [7]:
sheet1['supplier_product_code'] = sheet1['Cat No']

In [8]:
sheet1.head(3)

,Notes,Old Cat No,Cat No,Ships Directly from Japan office,2023 web price,2023 AMSbio price,2023 new discount from web price,Size,Units,Product Name,...,Reactivity,Product URL Full Path,Vendor_Website_Link,SKU_Master,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Lowest_Node,supplier_product_code
0,NaN,NaN,CAC-NU-07-001,NaN,563.0,422.3,0.25,200,µl,Anti Chondroitin Sulfate A (Chondroitin-4-Sulf...,...,All,https://www.cosmobiousa.com/products/anti-chon...,http://export.cosmobiousa.com/contents/detail....,CAC,NU,7,1,NaN,0,CAC-NU-07-001
1,NaN,NaN,CAC-NU-07-002,NaN,524.0,393.8,0.25,200,µl,Anti Neurocan Core Protein (Chondroitin Sulfat...,...,Rat,https://www.cosmobiousa.com/products/anti-neur...,http://export.cosmobiousa.com/contents/detail....,CAC,NU,7,2,NaN,0,CAC-NU-07-002
2,NaN,NaN,CAC-NU-07-003,NaN,524.0,393.8,0.25,200,µl,Anti Chondroitin Sulfate Proteoglycan 5 (Neuro...,...,"Rat, Other Animals",https://www.cosmobiousa.com/products/anti-neur...,http://export.cosmobiousa.com/contents/detail....,CAC,NU,7,3,NaN,0,CAC-NU-07-003


In [9]:
for i in range(len(sheet1)):
    sheet1.loc[i, 'Lowest_Node'] = fetchCategory(code=sheet1['supplier_product_code'].iloc[i], column = "supplier_product_code", cursor=cursor)
    
sheet1.head(3)

,Notes,Old Cat No,Cat No,Ships Directly from Japan office,2023 web price,2023 AMSbio price,2023 new discount from web price,Size,Units,Product Name,...,Reactivity,Product URL Full Path,Vendor_Website_Link,SKU_Master,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Lowest_Node,supplier_product_code
0,NaN,NaN,CAC-NU-07-001,NaN,563.0,422.3,0.25,200,µl,Anti Chondroitin Sulfate A (Chondroitin-4-Sulf...,...,All,https://www.cosmobiousa.com/products/anti-chon...,http://export.cosmobiousa.com/contents/detail....,CAC,NU,7,1,NaN,118,CAC-NU-07-001
1,NaN,NaN,CAC-NU-07-002,NaN,524.0,393.8,0.25,200,µl,Anti Neurocan Core Protein (Chondroitin Sulfat...,...,Rat,https://www.cosmobiousa.com/products/anti-neur...,http://export.cosmobiousa.com/contents/detail....,CAC,NU,7,2,NaN,118,CAC-NU-07-002
2,NaN,NaN,CAC-NU-07-003,NaN,524.0,393.8,0.25,200,µl,Anti Chondroitin Sulfate Proteoglycan 5 (Neuro...,...,"Rat, Other Animals",https://www.cosmobiousa.com/products/anti-neur...,http://export.cosmobiousa.com/contents/detail....,CAC,NU,7,3,NaN,118,CAC-NU-07-003


In [10]:
Categories_Not_Available =  sheet1[(sheet1['Lowest_Node'] == "NA") | (sheet1['Lowest_Node'] == 0)]
Categories_available = sheet1[(sheet1['Lowest_Node'] != 'NA') & (sheet1['Lowest_Node'] != 0)]

In [11]:
lowest_to_cat = pd.read_csv('lowest_nodes.csv',header=0)
lowest_to_cat.columns = ['Lowest_Node', 'Category']

In [12]:
sheet1.head(3)

,Notes,Old Cat No,Cat No,Ships Directly from Japan office,2023 web price,2023 AMSbio price,2023 new discount from web price,Size,Units,Product Name,...,Reactivity,Product URL Full Path,Vendor_Website_Link,SKU_Master,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Lowest_Node,supplier_product_code
0,NaN,NaN,CAC-NU-07-001,NaN,563.0,422.3,0.25,200,µl,Anti Chondroitin Sulfate A (Chondroitin-4-Sulf...,...,All,https://www.cosmobiousa.com/products/anti-chon...,http://export.cosmobiousa.com/contents/detail....,CAC,NU,7,1,NaN,118,CAC-NU-07-001
1,NaN,NaN,CAC-NU-07-002,NaN,524.0,393.8,0.25,200,µl,Anti Neurocan Core Protein (Chondroitin Sulfat...,...,Rat,https://www.cosmobiousa.com/products/anti-neur...,http://export.cosmobiousa.com/contents/detail....,CAC,NU,7,2,NaN,118,CAC-NU-07-002
2,NaN,NaN,CAC-NU-07-003,NaN,524.0,393.8,0.25,200,µl,Anti Chondroitin Sulfate Proteoglycan 5 (Neuro...,...,"Rat, Other Animals",https://www.cosmobiousa.com/products/anti-neur...,http://export.cosmobiousa.com/contents/detail....,CAC,NU,7,3,NaN,118,CAC-NU-07-003


object
int64


In [14]:

Categories_available = pd.merge(sheet1, lowest_to_cat, on='Lowest_Node', how='left')

In [15]:
Categories_available

,Notes,Old Cat No,Cat No,Ships Directly from Japan office,2023 web price,2023 AMSbio price,2023 new discount from web price,Size,Units,Product Name,...,Product URL Full Path,Vendor_Website_Link,SKU_Master,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Lowest_Node,supplier_product_code,Category
0,NaN,NaN,CAC-NU-07-001,NaN,563.0,422.3,0.25,200,µl,Anti Chondroitin Sulfate A (Chondroitin-4-Sulf...,...,https://www.cosmobiousa.com/products/anti-chon...,http://export.cosmobiousa.com/contents/detail....,CAC,NU,7,1,NaN,118,CAC-NU-07-001,13.0
1,NaN,NaN,CAC-NU-07-002,NaN,524.0,393.8,0.25,200,µl,Anti Neurocan Core Protein (Chondroitin Sulfat...,...,https://www.cosmobiousa.com/products/anti-neur...,http://export.cosmobiousa.com/contents/detail....,CAC,NU,7,2,NaN,118,CAC-NU-07-002,13.0
2,NaN,NaN,CAC-NU-07-003,NaN,524.0,393.8,0.25,200,µl,Anti Chondroitin Sulfate Proteoglycan 5 (Neuro...,...,https://www.cosmobiousa.com/products/anti-neur...,http://export.cosmobiousa.com/contents/detail....,CAC,NU,7,3,NaN,118,CAC-NU-07-003,13.0
3,NaN,NaN,CAC-NU-07-004,NaN,524.0,393.8,0.25,100,µl,Anti Syndecan-3 (N-Syndecan/Neuroglycan) pAb (...,...,https://www.cosmobiousa.com/products/anti-n-sy...,http://export.cosmobiousa.com/contents/detail....,CAC,NU,7,4,NaN,118,CAC-NU-07-004,13.0
4,NaN,NaN,CAC-NU-07-005,NaN,524.0,393.8,0.25,200,µl,Anti Neurocan Core Protein (Chondroitin Sulfat...,...,https://www.cosmobiousa.com/products/anti-neur...,http://export.cosmobiousa.com/contents/detail....,CAC,NU,7,5,NaN,118,CAC-NU-07-005,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,NaN,NaN,YMS-80049,NaN,574.0,508.5,0.11,25,Units,L-Glutamate Oxidase 25,...,https://www.cosmobiousa.com/products/l-glutama...,http://export.cosmobiousa.com/contents/detail....,YMS,80049,NaN,NaN,NaN,135,YMS-80049,16.0
204,NaN,NaN,YMS-80072-EX,NaN,2116.0,1697.3,0.20,1 x 96,rxns,Rat/Mouse Surfactant Protein D ELISA Kit,...,https://www.cosmobiousa.com/products/rat-mouse...,http://export.cosmobiousa.com/contents/detail....,YMS,80072,EX,NaN,NaN,142,YMS-80072-EX,17.0
205,NaN,NaN,YMS-80126-EX,NaN,2119.0,2025.1,0.04,96,Tests,S100A9 Assay Kit,...,https://www.cosmobiousa.com/products/s100a9-as...,http://export.cosmobiousa.com/products/detail....,YMS,80126,EX,NaN,NaN,142,YMS-80126-EX,17.0
206,NaN,NaN,YMS-80127-EX,NaN,2119.0,2025.1,0.04,66,Tests,OCT Assay Kit,...,https://www.cosmobiousa.com/products/oct-assay...,http://export.cosmobiousa.com/products/detail....,YMS,80127,EX,NaN,NaN,142,YMS-80127-EX,17.0


In [16]:
Categories_Not_Available

,Notes,Old Cat No,Cat No,Ships Directly from Japan office,2023 web price,2023 AMSbio price,2023 new discount from web price,Size,Units,Product Name,...,Reactivity,Product URL Full Path,Vendor_Website_Link,SKU_Master,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Lowest_Node,supplier_product_code
60,NEW,NaN,CSR-BRA-96P,NaN,390.666667,359.4,0.08,1,PLATE,BONE RESORPTION ASSAY PLATE 96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA,CSR-BRA-96P
61,NEW,NaN,CSR-BRA-96X2P,NaN,640.000000,588.8,0.08,2*1,PLATE,BONE RESORPTION ASSAY PLATE 96x2P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA,CSR-BRA-96X2P
62,NEW,NaN,CSR-BRA-S96P,NaN,462.666667,425.7,0.08,1,PLATE,BONE RESORPTION ASSAY PLATE S96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA,CSR-BRA-S96P
63,NEW,NaN,CSR-BRA-S96X2P,NaN,746.666667,686.9,0.08,2*1,PLATE,BONE RESORPTION ASSAY PLATE S96x2P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA,CSR-BRA-S96X2P
64,NEW,NaN,CSR-BRA-24X2KIT,NaN,1066.666667,981.3,0.08,2*1,KIT,BONE RESORPTION ASSAY KIT 24x2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA,CSR-BRA-24X2KIT
65,NEW,NaN,CSR-BRA-96KIT,NaN,746.666667,686.9,0.08,1,KIT,BONE RESORPTION ASSAY KIT 96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA,CSR-BRA-96KIT
66,NEW,NaN,CSR-BRA-96X2KIT,NaN,1209.333333,1112.6,0.08,2*1,KIT,BONE RESORPTION ASSAY KIT 96x2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA,CSR-BRA-96X2KIT
67,NEW,NaN,CSR-BRA-S96KIT,NaN,818.666667,753.2,0.08,1,KIT,BONE RESORPTION ASSAY KIT S96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA,CSR-BRA-S96KIT
68,NEW,NaN,CSR-BRA-S96X2KIT,NaN,1316.000000,1210.7,0.08,2*1,KIT,BONE RESORPTION ASSAY KIT S96x2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA,CSR-BRA-S96X2KIT
69,NEW,NaN,CSR-BRA-24X2P,NaN,498.666667,458.8,0.08,2*1,PLATE,BONE RESORPTION ASSAY PLATE 24x2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA,CSR-BRA-24X2P


In [ ]:
biorep = Categories_available[Categories_available['Category'] == 11]
molBio = Categories_available[Categories_available['Category'] == 12]
antibody = Categories_available[Categories_available['Category'] == 13]
ProtPep = Categories_available[Categories_available['Category'] == 14]
CellCull = Categories_available[Categories_available['Category'] == 15]
ReagLab = Categories_available[Categories_available['Category'] == 16]
KitsAssay = Categories_available[Categories_available['Category'] == 17]
BioSep = Categories_available[Categories_available['Category'] == 18]

In [ ]:
if len(biorep) > 0:
    biorep.to_excel('C://Users//Shri//AMSBio//AMSBIO Sharepoint - Documents//ShareDocs//Current Price_lists//2023//{}//Categorized//11BioRep.xlsx'.format(company), index=False)
if len(molBio) > 0:
    molBio.to_excel('C://Users//Shri//AMSBio//AMSBIO Sharepoint - Documents//ShareDocs//Current Price_lists//2023//{}//Categorized//12molBio.xlsx'.format(company), index=False)
if len(antibody) > 0:
    antibody.to_excel('C://Users//Shri//AMSBio//AMSBIO Sharepoint - Documents//ShareDocs//Current Price_lists//2023//{}//Categorized//13antibody.xlsx'.format(company), index=False)
if len(ProtPep) > 0:
    ProtPep.to_excel('C://Users//Shri//AMSBio//AMSBIO Sharepoint - Documents//ShareDocs//Current Price_lists//2023//{}//Categorized//14ProtPep.xlsx'.format(company), index=False)
if len(CellCull) > 0:
    CellCull.to_excel('C://Users//Shri//AMSBio//AMSBIO Sharepoint - Documents//ShareDocs//Current Price_lists//2023//{}//Categorized//15CellCull.xlsx'.format(company), index=False)
if len(ReagLab) > 0:
    ReagLab.to_excel('C://Users//Shri//AMSBio//AMSBIO Sharepoint - Documents//ShareDocs//Current Price_lists//2023//{}//Categorized//16ReagLab.xlsx'.format(company), index=False)
if len(KitsAssay) > 0:
    KitsAssay.to_excel('C://Users//Shri//AMSBio//AMSBIO Sharepoint - Documents//ShareDocs//Current Price_lists//2023//{}//Categorized//17KitsAssay.xlsx'.format(company), index=False)
if len(BioSep) > 0:
    BioSep.to_excel('C://Users//Shri//AMSBio//AMSBIO Sharepoint - Documents//ShareDocs//Current Price_lists//2023//{}//Categorized//18BioSep.xlsx'.format(company), index=False)

In [ ]:
Categories_available.to_excel('C://Users//Shri//AMSBio//AMSBIO Sharepoint - Documents//ShareDocs//Current Price_lists//2023//{}//Categorized//Categories_available.xlsx'.format(company), index=False)

In [ ]:
Categories_Not_Available.to_excel('C://Users//Shri//AMSBio//AMSBIO Sharepoint - Documents//ShareDocs//Current Price_lists//2023//{}//Categorized//Categories_not_available.xlsx'.format(company), index=False)